_Lambda School Data Science — Model Validation_ 

# Begin the modeling process

Objectives
- Train/Validate/Test split
- Cross-Validation
- Begin with baselines

## Get the Bank Marketing dataset

You have several ways you can get the dataset:

#### Kaggle
- Download from the [Kaggle competition page](https://www.kaggle.com/c/ds2-model-validation/data)
- Use the Kaggle API

#### GitHub
- Clone the [repo](https://github.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/tree/master/module-1-begin-modeling-process/bank-marketing)
- Download from the repo:

In [1]:
# !wget https://github.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-1-begin-modeling-process/bank-marketing/train_features.csv
# !wget https://github.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-1-begin-modeling-process/bank-marketing/train_labels.csv
# !wget https://github.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-1-begin-modeling-process/bank-marketing/test_features.csv
# !wget https://github.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/blob/master/module-1-begin-modeling-process/bank-marketing/sample_submission.csv    

In [3]:
%cd bank-marketing

[Errno 2] No such file or directory: 'bank-marketing'
/media/superio/Media/DATA_SCIENCE/DS-Unit-2-Sprint-4-Model-Validation/module-1-begin-modeling-process/bank-marketing


In [4]:
import pandas as pd

train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_features = pd.read_csv('test_features.csv')
sample_submission = pd.read_csv('sample_submission.csv')

train_features.shape, train_labels.shape, test_features.shape, sample_submission.shape

((30891, 20), (30891, 2), (10297, 20), (10297, 2))

In [5]:
y_train = train_labels['y']
X_train = train_features.drop(columns='id')
X_test  = test_features.drop(columns='id')

## Train/Validation/Test split

How can we get from a two-way split, to a three-way split?

We can use the [**`sklearn.model_selection.train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to split the training data into training and validation data.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((23168, 19), (7723, 19), (23168,), (7723,))

Fit on the training set.

Predict and score with the validation set.

## Majority class baseline

Determine the majority class:

In [7]:
y_train.value_counts(normalize=True)

0    0.886956
1    0.113044
Name: y, dtype: float64

Guess the majority class for every prediction:

In [8]:
majority_class = 0
y_pred = [majority_class] * len(y_val)

#### [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

Baseline accuracy by guessing the majority class for every prediction:

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)
# why is this different from y_train.value_counts(normalize=True)?
# b/c of the mean

0.888514825844879

#### [`sklearn.metrics.roc_auc_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

Baseline "ROC AUC" score by guessing the majority class for every prediction:

In [10]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)
# 0.5 is when you are not sorting it well

0.5

## Fast first models

### Ignore rows/columns with nulls

This dataset doesn't have any nulls:

In [11]:
X_train.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
dtype: int64

### Ignore nonnumeric features

Here are the numeric features:

In [12]:
import numpy as np
X_train.describe(include=np.number)

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000,23168.000000
mean,39.985238,2.546314,960.771323,0.176407,0.078583,93.576056,-40.512289,3.614938,5166.743828
std,10.414878,2.740707,191.046275,0.501047,1.572636,0.578511,4.629618,1.738414,72.542865
min,17.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,91.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


Here are the nonnumeric features:

In [13]:
X_train.describe(exclude=np.number)

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome
count,23168,23168,23168,23168,23168,23168,23168,23168,23168,23168
unique,12,4,8,3,3,3,2,10,5,3
top,admin.,married,university.degree,no,yes,no,cellular,may,thu,nonexistent
freq,5825,14017,6881,18297,12150,19121,14693,7764,4815,19955


Just select the nonnumeric features:

In [14]:
X_train_numeric = X_train.select_dtypes(np.number)
X_val_numeric = X_val.select_dtypes(np.number)

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_numeric, y_train)

y_pred = model.predict(X_val_numeric)
roc_auc_score(y_val, y_pred)

0.5823695850940274

### With Scaler

In [16]:
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_numeric)
X_val_scaled = scaler.transform(X_val_numeric)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_numeric, y_train)

y_pred = model.predict(X_val_scaled)
roc_auc_score(y_val, y_pred)
# worst roc_score than without scale, so it didn't really make a different

0.4820083402982508

### Same, as a pipeline

In [17]:
# Same method above cell, but return different roc_auc_score due random seed
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

pipeline.fit(X_train_numeric, y_train)

y_pred = pipeline.predict(X_val_numeric)
roc_auc_score(y_val, y_pred)

0.5824424501479475

### Encode "low cardinality" categoricals

One-hot encode the "low cardinality" categoricals

In [18]:
X_train.select_dtypes(exclude=np.number).nunique()

job            12
marital         4
education       8
default         3
housing         3
loan            3
contact         2
month          10
day_of_week     5
poutcome        3
dtype: int64

Install the Category Encoder library

If you're running on Google Colab:

```
!pip install category_encoders
```

If you're running locally with Anaconda:

```
!conda install -c conda-forge category_encoders
```

In [19]:
import category_encoders as ce

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)
roc_auc_score(y_val, y_pred)

0.606894557412077

In [20]:
X_val.shape, X_test.shape

((7723, 19), (10297, 19))

In [21]:
print(pd.Series(pipeline.predict(X_test)).value_counts())
# this is the probability correspond each class
pipeline.predict_proba(X_test)
# get the values of 2nd column, which we want
pipeline.predict_proba(X_test)[:, 1:]

# start submission to kaggle
submission = sample_submission.copy()
submission['y'] = pipeline.predict_proba(X_test)[:, 1:]
submission.to_csv('jp-submission-001.csv', index=False) # b/c we don't want to include index value in csv file
# upload submission-001.csv to Kaggle https://www.kaggle.com/c/ds2-model-validation/submit
submission.y.describe()

0    9919
1     378
dtype: int64


count    10297.000000
mean         0.111469
std          0.144938
min          0.006006
25%          0.039229
50%          0.057884
75%          0.093818
max          0.897442
Name: y, dtype: float64

In [22]:
!ls -lh *.csv

-rw-r--r-- 1 superio superio 262K Mar  4 16:54 jp-submission-001.csv
-rw-r--r-- 1 superio superio 101K Mar  4 13:10 sample_submission.csv
-rw-r--r-- 1 superio superio 1.3M Mar  4 13:10 test_features.csv
-rw-r--r-- 1 superio superio 3.7M Mar  4 13:10 train_features.csv
-rw-r--r-- 1 superio superio 242K Mar  4 13:10 train_labels.csv


In [59]:
!cat jp*.csv | xargs -l echo | wc -l

10298
